In [20]:
from pprint import pprint
from datetime import datetime
from itertools import combinations

import numpy as np
import pandas as pd
import math
from scipy import constants
from mendeleev import element


#INSTALAR GLAS COM !pip install --upgrade git+git://github.com/drcassar/glas@dev3

from glas import GlassSearcher as Searcher
from glas.constraint import Constraint
from glas.predict import Predict

In [21]:
###############################################################################
#                                   Searcher                                  #
##############################################################################+

class SearcherModified(Searcher):
    def __init__(self, config, design, constraints={}):
        super().__init__(
            config=config,
            design=design,
            constraints=constraints,
        )

    def report_dict(self, individual, verbose=True):
        report_dict = {}
        ind_array = np.array(individual).reshape(1,-1)

        pop_dict = {
            'population_array': ind_array,
            'population_weight': self.population_to_weight(ind_array),
            'atomic_array': self.population_to_atomic_array(ind_array),
        }

        if verbose:
            pprint(self.ind_to_dict(individual))

        if verbose:
            print()
            print('Predicted properties of this individual:')

        for ID in self.models:
            y_pred = self.models[ID].predict(pop_dict)[0]
            report_dict[ID] = y_pred
            if verbose:
                print(f'{self.design[ID]["name"]} = {y_pred:.3f} '
                      f'{self.design[ID].get("unit", "")}')

        for ID in self.report:
            y_pred = self.report[ID].predict(pop_dict)[0]
            within_domain = self.report[ID].is_within_domain(pop_dict)[0]
            if within_domain:
                report_dict[ID] = y_pred
                if verbose:
                    print(f'{self.design[ID]["name"]} = {y_pred:.3f} '
                        f'{self.design[ID].get("unit", "")}')
            elif verbose:
                print(f'{self.design[ID]["name"]} = Out of domain')
        
        dict_functions = {
           'VEC': parVEC, 
           'phi': parPhi,
           'omega': parOmega,
           'delta': parDelta,
           'H hydrogen solution': parHinf,
           'H hydride formation': parHf,
           'molar mass': parMM,
           'H phase transition': parHM,
           'H plateau': parHplat
          } 
        for ID in constraints:
            if ID == 'complexity' or ID == 'elements':
                pass
            else:
                print(f'{ID} = %.3f' % dict_functions[ID](ind_array)[0])
        
        if verbose:
            print()
            print()

        return report_dict



In [22]:
###############################################################################
#            Criação dos Dataframes com Valor de Propriedades                 #
##############################################################################+


dfProperties = pd.read_excel(r"properties.xlsx", index_col=0)
atomicSize = dfProperties["atomic size"] * 100
meltingTemperature = dfProperties["Tm (K)"]
dfHfHinf = pd.read_excel(r"Hinf_Hf.xlsx", index_col=0)
dfHmix = pd.read_excel(r"Hmix.xlsx", index_col=0)



hi_values = pd.read_csv('./hi.csv', sep = ';').to_numpy() #importa os valores de hi do arquivo csv e transforma em uma matriz 
bonding_alpha = pd.read_csv('./bonding_alpha.csv', sep = ';').to_numpy() #importa os valores da energia de ligação da estrutura alpha
bonding_delta = pd.read_csv('./bonding_delta.csv', sep = ';').to_numpy()#importa os valores da energia de ligação da estrutura delta


In [23]:
###############################################################################
#                           Variáveis Internas                                #
##############################################################################+


elements = np.array([
    "Mg",
    "Al",
    "Ti",
    "V",
    "Cr",
    "Mn",
    "Fe",
    "Co",
    "Ni",
    "Cu",
    "Zn",
    "Zr",
    "Nb",
    "Mo",
    "Pd",
    #"La",
    "Hf",
    "Ta",
    "W",
])

allMolarMass = {element(i).symbol: element(i).mass for i in elements}
arrMolarMass = np.array(list(allMolarMass.values()))

allVEC = {element(i).symbol: element(i).nvalence() for i in elements}
arrVEC = np.array(list(allVEC.values()))

allHinf = {element(i).symbol: dfHfHinf[i][0] for i in elements}
arrHinf = np.array(list(allHinf.values()))

allHf = {element(i).symbol: dfHfHinf[i][1] for i in elements}
arrHf = np.array(list(allHf.values()))

allMeltingT = {element(i).symbol: meltingTemperature[i] for i in elements}
arrMeltingT = np.array(list(allMeltingT.values()))

allAtomicSize = {element(i).symbol: atomicSize[i] for i in elements}
arrAtomicSize = np.array(list(allAtomicSize.values()))



In [24]:
###############################################################################
#                             Funções Vetorizadas                             #
##############################################################################+

def normalizer(compositions):
    arraySum = np.sum(compositions, axis=1)
    normValues = compositions / arraySum[:, None]
    return normValues


def Smix(compNorm):
    x = np.sum(np.nan_to_num((compNorm) * np.log(compNorm)), axis=1)
    Smix = -constants.R * 10 ** -3 * x
    return Smix


def Tm(compNorm):
    Tm = np.sum(compNorm * arrMeltingT, axis=1)
    return Tm


def Hmix(compNorm):
    elements_present = compNorm.sum(axis=0).astype(bool)
    compNorm = compNorm[:, elements_present]
    element_names = elements[elements_present]
    Hmix = np.zeros(compNorm.shape[0])
    for i, j in combinations(range(len(element_names)), 2):
        Hmix = (
            Hmix
            + 4
            * dfHmix[element_names[i]][element_names[j]]
            * compNorm[:, i]
            * compNorm[:, j]
        )
        
    return Hmix


def Sh(compNorm):
    Sh = abs(Hmix(compNorm)) / Tm(compNorm)
    return Sh


def csi_i(compNorm, AP):
    supportValue = np.sum((1/6)*math.pi*(arrAtomicSize*2)**3*compNorm, axis=1)
    rho = AP/supportValue
    csi_i = (1/6)*math.pi*rho[:, None]*(arrAtomicSize*2)**3*compNorm
    return csi_i


def deltaij(i, j, newCompNorm, newArrAtomicSize, csi_i_newCompNorm, AP):
    element1Size = newArrAtomicSize[i]*2
    element2Size = newArrAtomicSize[j]*2
    deltaij = ((csi_i_newCompNorm[:,i]*csi_i_newCompNorm[:,j])**(1/2)/AP)*(((element1Size - element2Size)**2)/(element1Size * element2Size))*(newCompNorm[:,i] * newCompNorm[:,j])**(1/2)
    return deltaij


def y1_y2(compNorm, AP):
    csi_i_compNorm = csi_i(compNorm, AP)
    elements_present = compNorm.sum(axis=0).astype(bool)
    newCompNorm = compNorm[:, elements_present]
    newCsi_i_compNorm = csi_i_compNorm[:, elements_present]
    newArrAtomicSize = arrAtomicSize[elements_present]
    y1 = np.zeros(newCompNorm.shape[0])
    y2 = np.zeros(newCompNorm.shape[0])
    for i, j in combinations(range(len(newCompNorm[0])), 2):
        deltaijValue = deltaij(i, j, newCompNorm, newArrAtomicSize, newCsi_i_compNorm, AP)
        y1 += deltaijValue * (newArrAtomicSize[i]*2 + newArrAtomicSize[j]*2) * (newArrAtomicSize[i]*2*newArrAtomicSize[j]*2)**(-1/2)
        y2_ = np.sum((newCsi_i_compNorm/AP) * (((newArrAtomicSize[i]*2*newArrAtomicSize[j]*2)**(1/2)) / (newArrAtomicSize*2)), axis=1)
        y2 += deltaijValue * y2_
    return y1, y2
    

def y3(compNorm, AP):
    csi_i_compNorm = csi_i(compNorm, AP)
    x = (csi_i_compNorm/AP)**(2/3)*compNorm**(1/3)
    y3 = (np.sum(x, axis=1))**3
    return y3


def Z(compNorm, AP):
    y1Values, y2Values = y1_y2(compNorm, AP)
    y3Values = y3(compNorm, AP)
    Z = ((1+AP+AP**2) - 3*AP*(y1Values+y2Values*AP) - AP**3*y3Values) * (1-AP)**(-3)
    return Z


def eq4B(compNorm, AP):
    y1Values, y2Values = y1_y2(compNorm, AP)
    y3Values = y3(compNorm, AP)
    eq4B = -(3/2) * (1-y1Values+y2Values+y3Values) + (3*y2Values+2*y3Values) * (1-AP)**-1 + (3/2) * (1-y1Values-y2Values-(1/3)*y3Values) * (1-AP)**-2 + (y3Values-1) * np.log(1-AP)
    return eq4B


def Se(compNorm, AP):
    Se = (eq4B(compNorm, AP) - np.log(Z(compNorm, AP)) - (3-2*AP) * (1-AP)**-2 + 3 + np.log((1+AP+AP**2-AP**3) * (1-AP)**-3)) * constants.R*10**-3
    return Se


def parDelta(compositions):
    compNorm = normalizer(compositions)
    atomicSizeMean = np.sum(compNorm * arrAtomicSize, axis=1)
    delta = (np.sum(compNorm*(1-(arrAtomicSize/atomicSizeMean[:, None]))**2, axis=1))**(1/2)
    return delta


def parHinf(compositions):
    compNorm = normalizer(compositions)
    Hinf = compNorm * arrHinf
    HinfFinal = np.sum(Hinf, axis=1)
    return HinfFinal


def parHf(compositions):
    compNorm = normalizer(compositions)
    Hf = compNorm * arrHf
    HfFinal = np.sum(Hf, axis=1)
    return HfFinal


def parVEC(compositions):
    compNorm = normalizer(compositions)
    VEC = compNorm * arrVEC
    VECFinal = np.sum(VEC, axis=1)
    return VECFinal


def parOmega(compositions):  
    compNorm = normalizer(compositions)
    omega = (Tm(compNorm) * Smix(compNorm)) / abs(Hmix(compNorm))
    return omega


def parMM(compositions):
    compNorm = normalizer(compositions)
    MMElements = compNorm * arrMolarMass
    MMFinal = np.sum(MMElements, axis=1)
    return MMFinal


def parPhi(compositions):
    #print(len(compositions),compositions)
    compNorm = normalizer(compositions)
    #print(len(compNorm),compNorm)
    SeBCC = Se(compNorm, 0.68)
    SeFCC = Se(compNorm, 0.74)
    SeMean = (abs(SeBCC) + abs(SeFCC)) / 2
    phi = (Smix(compNorm) - Sh(compNorm)) / SeMean
    #print(len(phi),phi)
    return phi


'''def parHM(compositions):  #determines the Enthalpy due the phase transition from the reference -- H_M diff from 0 only for delta phase
    compNorm = normalizer(compositions)
    HM = []
    for comp in compNorm:
    
        alpha_vetor = np.zeros([len(bonding_alpha),len(bonding_alpha)])
        for i in elements:
            for j in elements:
                alpha_vetor[np.where(bonding_alpha==i)[0],np.where(bonding_alpha==j)[0]]=comp[np.where(elements==i)[0]]*comp[np.where(elements==j)[0]]

        average_bond_alpha = alpha_vetor * bonding_alpha[0:,1:]
        average_bond_alpha = sum(sum(average_bond_alpha))

        E_total_alpha = average_bond_alpha * 4


        delta_vetor = np.zeros([len(bonding_delta),len(bonding_delta)])

        for i in elements:
            for j in elements:
                delta_vetor[np.where(bonding_delta==i)[0],np.where(bonding_delta==j)[0]]=comp[np.where(elements==i)[0]]*comp[np.where(elements==j)[0]]

        average_bond_delta = delta_vetor * bonding_delta[0:,1:]
        average_bond_delta = sum(sum(average_bond_delta))
        E_total_delta = average_bond_delta * 6

        H_delta = E_total_delta - E_total_alpha
        HM.append(H_delta)
    return np.array(HM)'''


def parHM(compositions):  #determines the Enthalpy due the phase transition from the reference -- H_M diff from 0 only for delta phase
    compNorm = normalizer(compositions)

    alpha_tensor = np.zeros([len(compNorm),len(bonding_alpha),len(bonding_alpha)],dtype=float)
    delta_tensor = np.zeros([len(compNorm),len(bonding_delta),len(bonding_delta)],dtype=float)
    
    
    for i in elements:
        for j in elements:
            alpha_tensor[0:,np.where(bonding_alpha==i)[0],np.where(bonding_alpha==j)[0]]=compNorm[0:,np.where(elements==i)[0]]*compNorm[0:,np.where(elements==j)[0]]
            delta_tensor[0:,np.where(bonding_delta==i)[0],np.where(bonding_delta==j)[0]]=compNorm[0:,np.where(elements==i)[0]]*compNorm[0:,np.where(elements==j)[0]]

    average_bond_alpha = alpha_tensor * bonding_alpha[0:,1:]
    average_bond_alpha = np.sum(np.sum(average_bond_alpha,axis=1),axis=1)
    E_total_alpha = average_bond_alpha * 4
    
    average_bond_delta = delta_tensor * bonding_delta[0:,1:]
    average_bond_delta = np.sum(np.sum(average_bond_delta,axis=1),axis=1)
    E_total_delta = average_bond_delta * 6
    
    HM = (E_total_delta - E_total_alpha).astype('float64')
    #HM = HM.astype('float64')

    '''for i in elements:
        for j in elements:
            delta_vetor[np.where(bonding_delta==i)[0],np.where(bonding_delta==j)[0]]=comp[np.where(elements==i)[0]]*comp[np.where(elements==j)[0]]

    average_bond_delta = delta_vetor * bonding_delta[0:,1:]
    average_bond_delta = sum(sum(average_bond_delta))
    E_total_delta = average_bond_delta * 6

    H_delta = E_total_delta - E_total_alpha
    HM.append(H_delta)'''
    return HM

def parHplat(compositions):
    compNorm = normalizer(compositions)
#    print(compNorm)
    plat_tensor = np.zeros([len(compNorm),len(hi_values),len(hi_values)],dtype=float) #cria um vetor de zeros com a mesma quantidade de coordenadas que entradas de elementos na matriz hi_values
#   composition_vetor = np.zeros(len(hi_values)) #cria um vetor de zeros com a mesma quantidade de coordenadas que entradas de elementos na matriz hi_values
    for i in elements:
        for j in elements:
            plat_tensor[0:,np.where(hi_values==i)[0],np.where(hi_values==j)[0]]=compNorm[0:,np.where(elements==i)[0]]*compNorm[0:,np.where(elements==j)[0]]          
#           composition_vetor[np.where(hi_values==i)[0]] = compNorm[i]        
    ha = plat_tensor[0:,0:,1] * hi_values[0:,1] #corrigi aqui, estava somando todas as matrizes
    ha = np.sum(ha,axis=1)
    
    hd = plat_tensor[0:,0:,3] * hi_values[0:,3]
    hd = np.sum(hd,axis=1)
    #print(len(ha),len(hd),len(plat_tensor))
    Hplat=(2*hd - ha + parHM(compositions)).astype('float64')
    return Hplat


In [25]:
###############################################################################
#                             Classes Tipo Predict                            #
##############################################################################+

class PredictMM(Predict):
    def __init__(self, all_elements, **kwargs):
        super().__init__()
        self.domain = {el: [0,1] for el in all_elements}

    def predict(self, population_dict):
        value = parMM(population_dict['population_array'])
        return value

    def get_domain(self):
        return self.domain

    def is_within_domain(self, population_dict):
        return np.ones(len(population_dict['population_array'])).astype(bool)
    

class PredictHinf(Predict):
    def __init__(self, all_elements, **kwargs):
        super().__init__()
        self.domain = {el: [0,1] for el in all_elements}

    def predict(self, population_dict):
        value = parHinf(population_dict['population_array'])
        return value

    def get_domain(self):
        return self.domain

    def is_within_domain(self, population_dict):
        return np.ones(len(population_dict['population_array'])).astype(bool)
    
    
class PredictHf(Predict):
    def __init__(self, all_elements, **kwargs):
        super().__init__()
        self.domain = {el: [0,1] for el in all_elements}

    def predict(self, population_dict):
        value = parHf(population_dict['population_array'])
        return value

    def get_domain(self):
        return self.domain

    def is_within_domain(self, population_dict):
        return np.ones(len(population_dict['population_array'])).astype(bool)

    
class PredictHplat(Predict):
    def __init__(self, all_elements, **kwargs):
        super().__init__()
        self.domain = {el: [0,1] for el in all_elements}

    def predict(self, population_dict):
        #value = parHf(population_dict['population_array']) precisava trocar aqui para parHplat
        value = parHplat(population_dict['population_array'])
        return value

    def get_domain(self):
        return self.domain

    def is_within_domain(self, population_dict):
        return np.ones(len(population_dict['population_array'])).astype(bool)


In [26]:
###############################################################################
#                           Classes Tipo Constraint                           #
##############################################################################+

class ConstraintPhi(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parPhi(population_dict['population_array'])
        bad = value <= self.config['min']

        distance_min = self.config['min'] - value
        distance = np.zeros(len(value))
        distance[bad] += distance_min[bad]

        penalty = bad * base_penalty + distance**2
        return penalty


class ConstraintComplexity(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        num_elements = population_dict['population_array'].astype(bool).sum(axis=1)
        logic1 = num_elements < self.config['min_elements']
        logic2 = num_elements > self.config['max_elements']
        bad = np.logical_or(logic1, logic2)

        distance_min = self.config['min_elements'] - num_elements
        distance_max = num_elements - self.config['max_elements']

        distance = np.zeros(len(num_elements))
        distance[logic1] += distance_min[logic1]
        distance[logic2] += distance_max[logic2]

        penalty = bad * base_penalty + distance**2
        return penalty
    

class ConstraintOmega(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parOmega(population_dict['population_array'])
        bad = value <= self.config['min']

        distance_min = self.config['min'] - value
        distance = np.zeros(len(value))
        distance[bad] += distance_min[bad]

        penalty = bad * base_penalty + distance**2
        return penalty
    
    
class ConstraintDelta(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parDelta(population_dict['population_array'])
        bad = value >= self.config['max']

        distance_max = value - self.config['max']
        distance = np.zeros(len(value))
        distance[bad] += distance_max[bad]

        penalty = bad * base_penalty + distance**2
        return penalty
    
    
class ConstraintVEC(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parVEC(population_dict['population_array'])
        bad = value >= self.config['max']

        distance_max = value - self.config['max']
        distance = np.zeros(len(value))
        distance[bad] += distance_max[bad]

        penalty = bad * base_penalty + distance**2
        return penalty
    
    
class ConstraintMM(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parMM(population_dict['population_array'])
        bad = value >= self.config['max']

        distance_max = value - self.config['max']
        distance = np.zeros(len(value))
        distance[bad] += distance_max[bad]

        penalty = bad * base_penalty + distance**2
        return penalty
    

class ConstraintHinf(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parHinf(population_dict['population_array'])
        bad = value >= self.config['max']

        distance_max = value - self.config['max']
        distance = np.zeros(len(value))
        distance[bad] += distance_max[bad]

        penalty = bad * base_penalty + distance**2
        return penalty
    
    
class ConstraintHf(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parHf(population_dict['population_array'])
        bad = value >= self.config['max']

        distance_max = value - self.config['max']
        distance = np.zeros(len(value))
        distance[bad] += distance_max[bad]

        penalty = bad * base_penalty + distance**2
        return penalty


class ConstraintElements(Constraint):
    def __init__(self, config, compound_list, **kwargs):
        super().__init__()
        self.config = config
        elemental_domain = {el: [0, 1] for el in compound_list}
        for el in config:
            elemental_domain[el] = config[el]
        self.elemental_domain = elemental_domain

    def compute(self, population_dict, base_penalty):
        norm_pop = normalizer(population_dict['population_array'])
        distance = np.zeros(population_dict['population_array'].shape[0])

        for n, el in enumerate(self.elemental_domain):
            el_atomic_frac = norm_pop[:, n]
            el_domain = self.elemental_domain.get(el, [0, 0])

            logic1 = el_atomic_frac > el_domain[1]
            distance[logic1] += el_atomic_frac[logic1] - el_domain[1]

            logic2 = el_atomic_frac < el_domain[0]
            distance[logic2] += el_domain[0] - el_atomic_frac[logic2]

        logic = distance > 0
        distance[logic] = (100 * distance[logic])**2 + base_penalty
        penalty = distance

        return penalty

class ConstraintHM(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parHM(population_dict['population_array'])
        bad = value <= self.config['min']

        distance_min = self.config['min'] - value
        distance = np.zeros(len(value))
        distance[bad] += distance_min[bad]
        penalty = bad * base_penalty + distance**2
        return penalty


class ConstraintHplat(Constraint):
    def __init__(self, config, **kwargs):
        super().__init__()
        self.config = config

    def compute(self, population_dict, base_penalty):
        value = parHplat(population_dict['population_array'])
        logic1 = value < self.config['min_plat'] #inverti os sinais, porque o valor é negativo
        logic2 = value > self.config['max_plat']
        bad = np.logical_or(logic1, logic2)

        distance_min = self.config['min_plat'] - value
        distance_max = value - self.config['max_plat']

        distance = np.zeros(len(value))
        distance[logic1] += distance_min[logic1]
        distance[logic2] += distance_max[logic2]

        penalty = bad * base_penalty + distance**2
        return penalty


In [27]:
###############################################################################
#                            Configuração de Busca                            #
##############################################################################+

design = {
    #'molar mass': {
    #    'class': PredictMM,
    #    'name': 'MM',
    #    'use_for_optimization': True,
    #    'config': {
    #        'min': 50,
    #        'max': 60,
    #        'objective': 'minimize',
    #        'weight': 1,
    #    }
    #},
    
    # 'H hydrogen solution': {
    #     'class': PredictHinf,
    #     'name': 'Hinf',
    #     'use_for_optimization': True,
    #     'config': {
    #         'min': -45,
    #         'max': -30,
    #         'objective': 'minimize',
    #         'weight': 5,
    #     }
    # },

    # 'H hydride formation': {
    #     'class': PredictHf,
    #     'name': 'Hf',
    #     'use_for_optimization': True,
    #     'config': {
    #         'min': -25,
    #         'max': -10,
    #         'objective': 'minimize',
    #         'weight': 1,
    #     }
    # },

#    'H plateau': {
#        'class': PredictHplat,
#        'name': 'Hplat',
#        'use_for_optimization': True,
#        'config': {
#            'min': -80,
#            'max': -60,
#            'objective': 'minimize',
#            'weight': 1,
#        }
#    },


}

constraints = {
    'elements': {
        'class': ConstraintElements,
        'config': {
            #'Mg': [0.10, 0.35],
            # 'Al': [0.0, 0.35],
            # 'Ti': [0.0, 0.35],
            # 'V': [0.2, 0.25],
            # 'Cr': [0.0, 0.35],
            # 'Mn': [0.0, 0.35],
            # 'Fe': [0.0, 0.35],
            # 'Co': [0.0, 0.35],
            # 'Ni': [0.0, 0.35],
            # 'Cu': [0.0, 0.35],
            # 'Zn': [0.0, 0.35],
            # 'Zr': [0.0, 0.35],
            # 'Nb': [0.2, 0.25],
            # 'Mo': [0.0, 0.35],
            # 'Pd': [0.0, 0.35],
            # 'La': [0.0, 0.35],
            # 'Hf': [0.0, 0.35],
            # 'Ta': [0.0, 0.35],
            # 'W': [0.0, 0.35],
        },
    },

    'phi': {
        'class': ConstraintPhi,
        'config': {
            'min': 10,
        },
    },

    # 'complexity': {
    #     'class': ConstraintComplexity,
    #     'config': {
    #         'min_elements': 6,
    #         'max_elements': 7,
    #     },
    # },
    
    # 'omega': {
    #     'class': ConstraintOmega,
    #     'config': {
    #         'min': 1.1,
    #     },
    # },

    # 'delta': {
    #     'class': ConstraintDelta,
    #     'config': {
    #         'max': 0.066,
    #     },
    # },

    'VEC': {
        'class': ConstraintVEC,
        'config': {
            'max': 6.87,
        },
    },

     'molar mass': {
         'class': ConstraintMM,
         'config': {
             'max': 65,
         },
     },

    #'H hydrogen solution': {
    #    'class': ConstraintHinf,
    #    'config': {
    #        'max': -20,
    #    },
    #},

    #'H hydride formation': {
    #    'class': ConstraintHf,
    #    'config': {
    #        'max': -40,
    #    },
    #},
    
    'H phase transition': {
        'class': ConstraintHM,
        'config': {
            'min': 0,
        },
    },

     'H plateau': {
         'class': ConstraintHplat,
         'config': {
             'min_plat': -30,
             'max_plat': -20,
         },
     },
        
}

config = {
    'num_generations': 2000,
    'population_size': 10000,
    'hall_of_fame_size': 1,
    'num_repetitions': 20,
    'compound_list': list(elements),
}



In [28]:
###############################################################################
#                                    Busca                                    #
##############################################################################+

all_hof = []
for _ in range(config['num_repetitions']):
    S = SearcherModified(config, design, constraints)
    S.start()
    S.run(config['num_generations'])
    all_hof.append(S.hof)


C:\Users\vtrds\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\vtrds\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\vtrds\AppData\Local\Temp\ipykernel_28496\3409325366.py:12: RuntimeWarning: divide by zero encountered in log
  x = np.sum(np.nan_to_num((compNorm) * np.log(compNorm)), axis=1)
C:\Users\vtrds\AppData\Local\Temp\ipykernel_28496\3409325366.py:12: RuntimeWarning: invalid value encountered in multiply
  x = np.sum(np.nan_to_num((compNorm) * np.log(compNorm)), axis=1)


Finished generation 001.  Best fitness is 2.67e+03. 
Finished generation 002.  Best fitness is 2.22e+03. 
Finished generation 003.  Best fitness is 2.18e+03. 
Finished generation 004.  Best fitness is 2.18e+03. 
Finished generation 005.  Best fitness is 2.16e+03. 
Finished generation 006.  Best fitness is 2.15e+03. 
Finished generation 007.  Best fitness is 2.15e+03. 
Finished generation 008.  Best fitness is 1.36e+03. 
Finished generation 009.  Best fitness is 1.36e+03. 
Finished generation 010.  Best fitness is 1.36e+03. 
Finished generation 011.  Best fitness is 1.36e+03. 
Finished generation 012.  Best fitness is 1.36e+03. 
Finished generation 013.  Best fitness is 1.36e+03. 
Finished generation 014.  Best fitness is 1.36e+03. 
Finished generation 015.  Best fitness is 1.36e+03. 
Finished generation 016.  Best fitness is 1.36e+03. 
Finished generation 017.  Best fitness is 1.36e+03. 
Finished generation 018.  Best fitness is 1.36e+03. 
Finished generation 019.  Best fitness is 1.35

Finished generation 156.  Best fitness is 1.01e+03. 
Finished generation 157.  Best fitness is 1.01e+03. 
Finished generation 158.  Best fitness is 1.01e+03. 
Finished generation 159.  Best fitness is 1.01e+03. 
Finished generation 160.  Best fitness is 1.01e+03. 
Finished generation 161.  Best fitness is 1.01e+03. 
Finished generation 162.  Best fitness is 1.01e+03. 
Finished generation 163.  Best fitness is 1.01e+03. 
Finished generation 164.  Best fitness is 1.01e+03. 
Finished generation 165.  Best fitness is 1.01e+03. 
Finished generation 166.  Best fitness is 1.01e+03. 
Finished generation 167.  Best fitness is 1.01e+03. 
Finished generation 168.  Best fitness is 1.01e+03. 
Finished generation 169.  Best fitness is 1.01e+03. 
Finished generation 170.  Best fitness is 1.01e+03. 
Finished generation 171.  Best fitness is 1.01e+03. 
Finished generation 172.  Best fitness is 1.01e+03. 
Finished generation 173.  Best fitness is 1.01e+03. 
Finished generation 174.  Best fitness is 1.01

Finished generation 311.  Best fitness is 1.01e+03. 
Finished generation 312.  Best fitness is 1.01e+03. 
Finished generation 313.  Best fitness is 1.01e+03. 
Finished generation 314.  Best fitness is 1.01e+03. 
Finished generation 315.  Best fitness is 1.01e+03. 
Finished generation 316.  Best fitness is 1.01e+03. 
Finished generation 317.  Best fitness is 1.01e+03. 
Finished generation 318.  Best fitness is 1.01e+03. 
Finished generation 319.  Best fitness is 1.01e+03. 
Finished generation 320.  Best fitness is 1.01e+03. 
Finished generation 321.  Best fitness is 1.01e+03. 
Finished generation 322.  Best fitness is 1.01e+03. 
Finished generation 323.  Best fitness is 1.01e+03. 
Finished generation 324.  Best fitness is 1.01e+03. 
Finished generation 325.  Best fitness is 1.01e+03. 
Finished generation 326.  Best fitness is 1.01e+03. 
Finished generation 327.  Best fitness is 1.01e+03. 
Finished generation 328.  Best fitness is 1.01e+03. 
Finished generation 329.  Best fitness is 1.01

Finished generation 466.  Best fitness is 1.01e+03. 
Finished generation 467.  Best fitness is 1.01e+03. 
Finished generation 468.  Best fitness is 1.01e+03. 
Finished generation 469.  Best fitness is 1.01e+03. 
Finished generation 470.  Best fitness is 1.01e+03. 
Finished generation 471.  Best fitness is 1.01e+03. 
Finished generation 472.  Best fitness is 1.01e+03. 
Finished generation 473.  Best fitness is 1.01e+03. 
Finished generation 474.  Best fitness is 1.01e+03. 
Finished generation 475.  Best fitness is 1.01e+03. 
Finished generation 476.  Best fitness is 1.01e+03. 
Finished generation 477.  Best fitness is 1.01e+03. 
Finished generation 478.  Best fitness is 1.01e+03. 
Finished generation 479.  Best fitness is 1.01e+03. 
Finished generation 480.  Best fitness is 1.01e+03. 
Finished generation 481.  Best fitness is 1.01e+03. 
Finished generation 482.  Best fitness is 1.01e+03. 
Finished generation 483.  Best fitness is 1.01e+03. 
Finished generation 484.  Best fitness is 1.01

...

Finished generation 1294.  Best fitness is 1.02e+03. 
Finished generation 1295.  Best fitness is 1.02e+03. 
Finished generation 1296.  Best fitness is 1.02e+03. 
Finished generation 1297.  Best fitness is 1.02e+03. 
Finished generation 1298.  Best fitness is 1.02e+03. 
Finished generation 1299.  Best fitness is 1.02e+03. 
Finished generation 1300.  Best fitness is 1.02e+03. 
Finished generation 1301.  Best fitness is 1.02e+03. 
Finished generation 1302.  Best fitness is 1.02e+03. 
Finished generation 1303.  Best fitness is 1.02e+03. 
Finished generation 1304.  Best fitness is 1.02e+03. 
Finished generation 1305.  Best fitness is 1.02e+03. 
Finished generation 1306.  Best fitness is 1.02e+03. 
Finished generation 1307.  Best fitness is 1.02e+03. 
Finished generation 1308.  Best fitness is 1.02e+03. 
Finished generation 1309.  Best fitness is 1.02e+03. 
Finished generation 1310.  Best fitness is 1.02e+03. 
Finished generation 1311.  Best fitness is 1.02e+03. 
Finished generation 1312.  B

Finished generation 1446.  Best fitness is 1.02e+03. 
Finished generation 1447.  Best fitness is 1.02e+03. 
Finished generation 1448.  Best fitness is 1.02e+03. 
Finished generation 1449.  Best fitness is 1.02e+03. 
Finished generation 1450.  Best fitness is 1.02e+03. 
Finished generation 1451.  Best fitness is 1.02e+03. 
Finished generation 1452.  Best fitness is 1.02e+03. 
Finished generation 1453.  Best fitness is 1.02e+03. 
Finished generation 1454.  Best fitness is 1.02e+03. 
Finished generation 1455.  Best fitness is 1.02e+03. 
Finished generation 1456.  Best fitness is 1.02e+03. 
Finished generation 1457.  Best fitness is 1.02e+03. 
Finished generation 1458.  Best fitness is 1.02e+03. 
Finished generation 1459.  Best fitness is 1.02e+03. 
Finished generation 1460.  Best fitness is 1.02e+03. 
Finished generation 1461.  Best fitness is 1.02e+03. 
Finished generation 1462.  Best fitness is 1.02e+03. 
Finished generation 1463.  Best fitness is 1.02e+03. 
Finished generation 1464.  B

Finished generation 1598.  Best fitness is 1.02e+03. 
Finished generation 1599.  Best fitness is 1.02e+03. 
Finished generation 1600.  Best fitness is 1.02e+03. 
Finished generation 1601.  Best fitness is 1.02e+03. 
Finished generation 1602.  Best fitness is 1.02e+03. 
Finished generation 1603.  Best fitness is 1.02e+03. 
Finished generation 1604.  Best fitness is 1.02e+03. 
Finished generation 1605.  Best fitness is 1.02e+03. 
Finished generation 1606.  Best fitness is 1.02e+03. 
Finished generation 1607.  Best fitness is 1.02e+03. 
Finished generation 1608.  Best fitness is 1.02e+03. 
Finished generation 1609.  Best fitness is 1.02e+03. 
Finished generation 1610.  Best fitness is 1.02e+03. 
Finished generation 1611.  Best fitness is 1.02e+03. 
Finished generation 1612.  Best fitness is 1.02e+03. 
Finished generation 1613.  Best fitness is 1.02e+03. 
Finished generation 1614.  Best fitness is 1.02e+03. 
Finished generation 1615.  Best fitness is 1.02e+03. 
Finished generation 1616.  B

Finished generation 1750.  Best fitness is 1.02e+03. 
Finished generation 1751.  Best fitness is 1.02e+03. 
Finished generation 1752.  Best fitness is 1.02e+03. 
Finished generation 1753.  Best fitness is 1.02e+03. 
Finished generation 1754.  Best fitness is 1.02e+03. 
Finished generation 1755.  Best fitness is 1.02e+03. 
Finished generation 1756.  Best fitness is 1.02e+03. 
Finished generation 1757.  Best fitness is 1.02e+03. 
Finished generation 1758.  Best fitness is 1.02e+03. 
Finished generation 1759.  Best fitness is 1.02e+03. 
Finished generation 1760.  Best fitness is 1.02e+03. 
Finished generation 1761.  Best fitness is 1.02e+03. 
Finished generation 1762.  Best fitness is 1.02e+03. 
Finished generation 1763.  Best fitness is 1.02e+03. 
Finished generation 1764.  Best fitness is 1.02e+03. 
Finished generation 1765.  Best fitness is 1.02e+03. 
Finished generation 1766.  Best fitness is 1.02e+03. 
Finished generation 1767.  Best fitness is 1.02e+03. 
Finished generation 1768.  B

Finished generation 1902.  Best fitness is 1.02e+03. 
Finished generation 1903.  Best fitness is 1.02e+03. 
Finished generation 1904.  Best fitness is 1.02e+03. 
Finished generation 1905.  Best fitness is 1.02e+03. 
Finished generation 1906.  Best fitness is 1.02e+03. 
Finished generation 1907.  Best fitness is 1.02e+03. 
Finished generation 1908.  Best fitness is 1.02e+03. 
Finished generation 1909.  Best fitness is 1.02e+03. 
Finished generation 1910.  Best fitness is 1.02e+03. 
Finished generation 1911.  Best fitness is 1.02e+03. 
Finished generation 1912.  Best fitness is 1.02e+03. 
Finished generation 1913.  Best fitness is 1.02e+03. 
Finished generation 1914.  Best fitness is 1.02e+03. 
Finished generation 1915.  Best fitness is 1.02e+03. 
Finished generation 1916.  Best fitness is 1.02e+03. 
Finished generation 1917.  Best fitness is 1.02e+03. 
Finished generation 1918.  Best fitness is 1.02e+03. 
Finished generation 1919.  Best fitness is 1.02e+03. 
Finished generation 1920.  B

In [29]:
###############################################################################
#                                    Report                                   #
##############################################################################+

print()
print('--------  REPORT -------------------')
print()
print('--------  Design Configuration -------------------')
pprint(config)
print()
pprint(design)
print()
print('--------  Constraints -------------------')
pprint(constraints)
print()

df_list = []
df_list2 = []

for p, hof in enumerate(all_hof):

    dfComp = pd.DataFrame(normalizer(hof), columns=list(elements))*100

    df_list.append(dfComp)
    
    dict_functions = {
           'VEC': parVEC, 
           'phi': parPhi,
           'omega': parOmega,
           'delta': parDelta,
           'H hydrogen solution': parHinf,
           'H hydride formation': parHf,
           'molar mass': parMM,
           'H phase transition': parHM,
           'H plateau': parHplat
          } 
        
    for ID in constraints:
        if ID == 'complexity' or ID == 'elements':
            pass
        else:
            dfComp = pd.DataFrame(dict_functions[ID](hof), columns=[ID])
            df_list2.append(dfComp)
            
            
    for ID in design:
         dfComp = pd.DataFrame(dict_functions[ID](hof), columns=[ID])
         df_list2.append(dfComp)
         

    print()
    print(f'------- RUN {p+1} -------------')
    print()
    for n, ind in enumerate(hof):
        print(f'Position {n+1} (mol%)')
        S.report_dict(ind, verbose=True)

dfComp = pd.concat(df_list, axis=0)

dfComp = dfComp.reset_index(drop=True)

dfPar = pd.concat(df_list2, axis=0)

for ID in design:
    dfPar2 = dfPar[ID].dropna().reset_index(drop=True)
    dfComp = dfComp.join(dfPar2)
    
for ID in constraints:
    if ID == 'complexity' or ID == 'elements':
        pass
    else:
        dfPar2 = dfPar[ID].dropna().reset_index(drop=True)
        dfComp = dfComp.join(dfPar2)

now = datetime.now()

dfComp.to_excel(f'{now.strftime("%d%m%Y_%H%M%S")}.xlsx')


--------  REPORT -------------------

--------  Design Configuration -------------------
{'compound_list': ['Mg',
                   'Al',
                   'Ti',
                   'V',
                   'Cr',
                   'Mn',
                   'Fe',
                   'Co',
                   'Ni',
                   'Cu',
                   'Zn',
                   'Zr',
                   'Nb',
                   'Mo',
                   'Pd',
                   'Hf',
                   'Ta',
                   'W'],
 'hall_of_fame_size': 1,
 'num_generations': 2000,
 'num_repetitions': 20,
 'population_size': 10000}

{}

--------  Constraints -------------------
{'H phase transition': {'class': <class '__main__.ConstraintHM'>,
                        'config': {'min': 0}},
 'H plateau': {'class': <class '__main__.ConstraintHplat'>,
               'config': {'max_plat': -20, 'min_plat': -30}},
 'VEC': {'class': <class '__main__.ConstraintVEC'>, 'config': {'max': 6.87}},

C:\Users\vtrds\AppData\Local\Temp\ipykernel_28496\3409325366.py:12: RuntimeWarning: divide by zero encountered in log
  x = np.sum(np.nan_to_num((compNorm) * np.log(compNorm)), axis=1)
C:\Users\vtrds\AppData\Local\Temp\ipykernel_28496\3409325366.py:12: RuntimeWarning: invalid value encountered in multiply
  x = np.sum(np.nan_to_num((compNorm) * np.log(compNorm)), axis=1)



------- RUN 2 -------------

Position 1 (mol%)
{'Al': 35.11, 'Cr': 33.33, 'Cu': 7.45, 'Mo': 17.73, 'V': 2.13, 'W': 4.26}

Predicted properties of this individual:
phi = 10.053
VEC = 5.298
molar mass = 57.456
H phase transition = -4.037
H plateau = -20.037



------- RUN 3 -------------

Position 1 (mol%)
{'Al': 1.99,
 'Cr': 13.43,
 'Cu': 1.0,
 'Fe': 0.5,
 'Mn': 4.98,
 'Mo': 14.43,
 'Nb': 1.99,
 'Pd': 0.5,
 'Ta': 1.0,
 'Ti': 49.75,
 'V': 1.0,
 'W': 2.49,
 'Zn': 3.48,
 'Zr': 3.48}

Predicted properties of this individual:
phi = 10.278
VEC = 5.174
molar mass = 63.536
H phase transition = 5.237
H plateau = -23.117



------- RUN 4 -------------

Position 1 (mol%)
{'Al': 5.51,
 'Cr': 1.27,
 'Fe': 2.12,
 'Hf': 0.42,
 'Mg': 5.08,
 'Mn': 5.08,
 'Mo': 3.39,
 'Nb': 0.85,
 'Ni': 0.42,
 'Ti': 41.53,
 'V': 28.81,
 'W': 0.85,
 'Zn': 4.24,
 'Zr': 0.42}

Predicted properties of this individual:
phi = 10.906
VEC = 4.852
molar mass = 51.674
H phase transition = 0.693
H plateau = -21.104



------- RUN 